
# 使用Selenium提取人民网搜索数据



![image.png](./images/people.png)

http://search.people.com.cn/cnpeople/news/getNewsResult.jsp

![image.png](./images/people2.png)


点击下一页页面的URL不变

![image.png](./images/people3.png)


鼠标右键查看页码

In [4]:
print(*range(1, 3))

1 2


In [5]:
url = 'http://search.people.com.cn/cnpeople/search.do?pageNum='
path = '&keyword=%D0%C2%B9%DA+%D6%D0%D2%BD&siteName=news&facetFlag=null&nodeType=belongsId&nodeId='
page_num = range(1, 3)
urls = [url+str(i)+path for i in page_num]
for i in urls[-3:]:
    print(i)
    

http://search.people.com.cn/cnpeople/search.do?pageNum=1&keyword=%D0%C2%B9%DA+%D6%D0%D2%BD&siteName=news&facetFlag=null&nodeType=belongsId&nodeId=
http://search.people.com.cn/cnpeople/search.do?pageNum=2&keyword=%D0%C2%B9%DA+%D6%D0%D2%BD&siteName=news&facetFlag=null&nodeType=belongsId&nodeId=


## 无法通过requests直接获取

提醒：您的访问可能对网站造成危险，已被云防护安全拦截

```
import requests
from bs4 import BeautifulSoup

content = requests.get(urls[0])
content.encoding = 'utf-8'

soup = BeautifulSoup(content.text, 'html.parser') 
soup
```

In [2]:
import requests
from bs4 import BeautifulSoup

content = requests.get(urls[0])
content.encoding = 'utf-8'
soup = BeautifulSoup(content.text, 'html.parser') 
soup

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<style>
body{ background:#eff1f0; font-family: microsoft yahei; color:#969696; font-size:12px;}
.online-desc-con { text-align:center; }
.r-tip01 { color: #969696; font-size: 16px; display: block; text-align: center; width: 500px; padding: 0 10px; overflow: hidden; text-overflow: ellipsis; margin: 0 auto 15px; }
.r-tip02 { color: #b1b0b0; font-size: 12px; display: block; margin-top: 20px; margin-bottom: 20px; }
.r-tip02 a:visited { text-decoration: underline; color: #0088CC; }
.r-tip02 a:link { text-decoration: underline; color: #0088CC; }
img { border: 0; }
</style>
<script>
void(function fuckie6(){if(location.hash && /MSIE 6/.test(navigator.userAgent) && !/jsl_sec/.test(location.href)){location.href = location.href.split('#')[0] + '&jsl_sec' + location.hash}})();
var error_403 = '';
if(error_403 == '') {
	error_403 = '当前访问疑似黑客攻击，已被创宇盾拦截。';
}
</script>
</head>
<body>
<div class="online-desc-con" style="width:550px;padding-top:15px;

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

browser = webdriver.Chrome()
dat = []
for k, j in enumerate(urls):
    print(k+1)
    time.sleep(1)
    browser.get(j) 
    source = browser.page_source
    soup = BeautifulSoup(source, 'html.parser') 
    d = soup.find_all('ul')
    while len(d) < 2:
        print(k+1, 'null error and retry')
        time.sleep(1)
        browser.get(j) 
        source = browser.page_source
        soup = BeautifulSoup(source, 'html.parser') 
        d = soup.find_all('ul')
        
    for i in d[1:]:
        urli = i.find('a')['href']
        title = i.find('a').text
        time_stamp = i.find_all('li')[-1].text.split('\xa0')[-1]
        dat.append([k+1, urli, title, time_stamp])

browser.close()
len(dat)

1
2


64

In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

browser = webdriver.Chrome()
dat = []
for k, j in enumerate(urls):
    print(k+1)
    time.sleep(1)
    browser.get(j) 
    source = browser.page_source
    soup = BeautifulSoup(source, 'html.parser') 
    d = soup.find_all('ul')
    while len(d) < 2:
        print(k+1, 'null error and retry')
        time.sleep(1)
        browser.get(j) 
        source = browser.page_source
        soup = BeautifulSoup(source, 'html.parser') 
        d = soup.find_all('ul')
        
    for i in d[1:]:
        urli = i.find('a')['href']
        title = i.find('a').text
        time_stamp = i.find_all('li')[-1].text.split('\xa0')[-1]
        dat.append([k+1, urli, title, time_stamp])

browser.close()
len(dat)

1
2
3
4
5
6
7
8
9
10
11
11 null error and retry
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


1416

In [14]:
import pandas as pd
df = pd.DataFrame(dat, columns = ['pagenum', 'url', 'title', 'time'])
df.head()

,pagenum,url,title,time
0,1,http://health.people.com.cn/n1/2020/0606/c1473...,[全国首部新型冠状病毒肺炎中医医案类专著在京举行新书首发仪式],2020-06-06 10:33:49
1,1,http://health.people.com.cn/n1/2020/0606/c1473...,为维护人民健康提供有力保障,2020-06-06 08:48:46
2,1,http://politics.people.com.cn/n1/2020/0606/c10...,为维护人民健康提供有力保障,2020-06-06 05:09:28
3,1,http://opinion.people.com.cn/n1/2020/0606/c100...,人民日报评论员：强化公共卫生体系的科技支撑,2020-06-06 05:07:22
4,1,http://health.people.com.cn/n1/2020/0605/c1473...,防控疫情，中医药发挥了重要作用,2020-06-05 08:36:27


In [16]:
len(df)

1416

In [17]:
df.to_csv('../data/people_com_search20200606.csv', index = False)

## Reading data with Pandas

In [20]:
with open('../data/people_com_search20200606.csv', 'r') as f:
    lines = f.readlines()
len(lines)

1423

In [21]:
import pandas as pd
df2 = pd.read_csv('../data/people_com_search20200606.csv')
df2.head()
len(df2)

1416

In [24]:
for i in df2['url'].tolist()[:10]:
    print(i)

http://health.people.com.cn/n1/2020/0606/c14739-31737564.html
http://health.people.com.cn/n1/2020/0606/c14739-31737424.html
http://politics.people.com.cn/n1/2020/0606/c1001-31737282.html
http://opinion.people.com.cn/n1/2020/0606/c1003-31737274.html
http://health.people.com.cn/n1/2020/0605/c14739-31736476.html
http://politics.people.com.cn/n1/2020/0605/c1001-31735966.html
http://world.people.com.cn/n1/2020/0604/c1002-31735815.html
http://www.people.com.cn/n1/2020/0604/c32306-31735734.html
http://cpc.people.com.cn/n1/2020/0604/c419242-31735244.html
http://health.people.com.cn/n1/2020/0604/c14739-31734772.html
